# LLM Optimization Modelling Experiment

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [159]:
problem = '''You are tasked with scheduling the power output of 6 electric power thermal units over the timespan of 15 periods. There is a constant startup cost of for each power unit that is applied if the power plant is turned on. The startup costs are 10324€, 5678€, 7802€, 12899€, 4596€ and 9076€ for powerplants 1 to 6, respectively. In addition, there is a constant shutdown cost for each power unit that is applied if the power plant is turned off. The shutdown costs are 2673€, 5893€, 982€, 6783€, 2596€ and 3561€ for powerplants 1 to 6, respectively. There is also a fixed and variable cost applied if the power plant is running. The fixed cost is constant and the variable cost is proportional to the output of a power plant. There are lower and upper bounds for the output power for each unit. The variable costs are different for each power plant and at each time step. Next, there are maximum power increments and decrements for each power plant that limit how much the output can change from one time period to the next. There is a total power demand that needs to be fulfilled by the power units. Finally, for security reasons, the total available power out should always be 10% higher than the demand. 
'''

## 2. Generate the mathematical model

In [177]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [178]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Power Plant Scheduling

**Parameters:**

*  **T:** Number of time periods (T = 15).
*  **N:** Number of power units (N = 6).
*  **SU_n:** Startup cost for power unit n (€).
*  **SD_n:** Shutdown cost for power unit n (€).
*  **FC_n:** Fixed cost for running power unit n (€).
*  **VC_{n,t}:** Variable cost per unit output for power unit n at time t (€/MWh).
*  **P_min_n:** Minimum power output for power unit n (MW).
*  **P_max_n:** Maximum power output for power unit n (MW).
*  **R_up_n:** Maximum power increment for power unit n from one time period to the next (MW).
*  **R_down_n:** Maximum power decrement for power unit n from one time period to the next (MW).
*  **D_t:** Total power demand at time t (MW).

**Decision Variables:**

*  **P_{n,t}:** Power output of unit n at time t (MW).
*  **U_{n,t}:** Binary variable indicating if unit n is started up at time t (1 if started, 0 otherwise).
*  **V_{n,t}:** Binary variable indicating if unit n is shut down at time t (1 if shut down, 0 otherwise).

**Objective Function:**

Minimize the total cost:

```
Minimize: ∑_{n=1}^{N} ∑_{t=1}^{T} (FC_n * U_{n,t} + VC_{n,t} * P_{n,t} + SU_n * U_{n,t} + SD_n * V_{n,t})
```

**Constraints:**

1. **Power balance:** Total power output should meet the demand: 
   ```
   ∑_{n=1}^{N} P_{n,t} ≥ 1.1 * D_t,  ∀ t ∈ {1, ..., T} 
   ```

2. **Power output limits:** Power output should be within the unit's limits:
   ```
   P_min_n ≤ P_{n,t} ≤ P_max_n,  ∀ n ∈ {1, ..., N},  ∀ t ∈ {1, ..., T}
   ```

3. **Ramp rate limits:** Power output changes should respect ramp rates:
   ```
   P_{n,t} - P_{n,t-1} ≤ R_up_n,  ∀ n ∈ {1, ..., N},  ∀ t ∈ {2, ..., T}
   P_{n,t-1} - P_{n,t} ≤ R_down_n,  ∀ n ∈ {1, ..., N},  ∀ t ∈ {2, ..., T}
   ```

4. **Startup and shutdown logic:**
   ```
   U_{n,t} - V_{n,t} = (P_{n,t} > 0) - (P_{n,t-1} > 0), ∀ n ∈ {1, ..., N},  ∀ t ∈ {2, ..., T}
   ```

**Parameter Values:**

* SU = [10324, 5678, 7802, 12899, 4596, 9076]
* SD = [2673, 5893, 982, 6783, 2596, 3561]
*  VC_{n,t}, FC_n, P_min_n, P_max_n, R_up_n, R_down_n, and D_t are assumed to be given as input data.

This mathematical optimization model captures the essence of the power plant scheduling problem. By solving this model, you can find the optimal power output for each unit at each time period to minimize the total cost while meeting the demand and operational constraints. 


## 3. Generate the pyomo code

In [179]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [180]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (Replace with actual data)
SU = [10324, 5678, 7802, 12899, 4596, 9076]  
SD = [2673, 5893, 982, 6783, 2596, 3561]
FC = [100, 80, 90, 120, 70, 110] # Sample data
VC = { (n,t): 0.5 + 0.1*n + 0.05*t for n in range(1,7) for t in range(1,16)} # Sample data
P_min = [50, 30, 40, 60, 20, 80] # Sample data
P_max = [200, 150, 180, 250, 100, 300] # Sample data
R_up = [40, 30, 35, 50, 25, 60] # Sample data
R_down = [40, 30, 35, 50, 25, 60] # Sample data
D = [400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 1050, 1100] # Sample data

# Model Initialization
model = pyo.ConcreteModel()

# Sets
model.T = pyo.RangeSet(1, 15) # Time periods
model.N = pyo.RangeSet(1, 6) # Power units

# Parameters
model.SU = pyo.Param(model.N, initialize=dict(enumerate(SU, 1))) # Startup cost
model.SD = pyo.Param(model.N, initialize=dict(enumerate(SD, 1))) # Shutdown cost
model.FC = pyo.Param(model.N, initialize=dict(enumerate(FC, 1))) # Fixed cost
model.VC = pyo.Param(model.N, model.T, initialize=VC) # Variable cost
model.P_min = pyo.Param(model.N, initialize=dict(enumerate(P_min, 1))) # Minimum power output
model.P_max = pyo.Param(model.N, initialize=dict(enumerate(P_max, 1))) # Maximum power output
model.R_up = pyo.Param(model.N, initialize=dict(enumerate(R_up, 1))) # Ramp-up rate
model.R_down = pyo.Param(model.N, initialize=dict(enumerate(R_down, 1))) # Ramp-down rate
model.D = pyo.Param(model.T, initialize=dict(enumerate(D, 1))) # Power demand

# Decision Variables
model.P = pyo.Var(model.N, model.T, within=pyo.NonNegativeReals) # Power output
model.U = pyo.Var(model.N, model.T, within=pyo.Binary) # Startup status
model.V = pyo.Var(model.N, model.T, within=pyo.Binary) # Shutdown status

# Objective Function
def objective_rule(model):
    return sum(model.FC[n] * model.U[n,t] + model.VC[n,t] * model.P[n,t] + model.SU[n] * model.U[n,t] + model.SD[n] * model.V[n,t] for n in model.N for t in model.T)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
# Power balance
def power_balance_rule(model, t):
    return sum(model.P[n,t] for n in model.N) >= 1.1 * model.D[t] # 1.1 represents a 10% safety margin
model.power_balance = pyo.Constraint(model.T, rule=power_balance_rule)

# Power output limits
def power_output_limits_rule(model, n, t):
    return (model.P_min[n], model.P[n,t], model.P_max[n])
model.power_output_limits = pyo.Constraint(model.N, model.T, rule=power_output_limits_rule)

# Ramp rate limits
def ramp_up_rule(model, n, t):
    if t > 1:
        return model.P[n,t] - model.P[n,t-1] <= model.R_up[n]
    else:
        return pyo.Constraint.Skip
model.ramp_up = pyo.Constraint(model.N, model.T, rule=ramp_up_rule)

def ramp_down_rule(model, n, t):
    if t > 1:
        return model.P[n,t-1] - model.P[n,t] <= model.R_down[n]
    else:
        return pyo.Constraint.Skip
model.ramp_down = pyo.Constraint(model.N, model.T, rule=ramp_down_rule)

# Startup and Shutdown logic
def startup_shutdown_rule(model, n, t):
    if t > 1:
        return model.U[n,t] - model.V[n,t] == (model.P[n,t] > 0) - (model.P[n,t-1] > 0)
    else:
        return pyo.Constraint.Skip
model.startup_shutdown = pyo.Constraint(model.N, model.T, rule=startup_shutdown_rule)

# Solve the model
solver = pyo.SolverFactory('cbc') # Replace 'cbc' with your preferred solver
results = solver.solve(model)

# Print results
print(results)

# Access and print specific variable values
for t in model.T:
    print(f"Time period {t}:")
    for n in model.N:
        print(f"  Unit {n} power output: {pyo.value(model.P[n,t])}")
```

**Explanation and Sample Data:**

*   **Sample Data:** I've filled in sample data for the parameters like `FC`, `VC`, `P_min`, `P_max`, `R_up`, `R_down`, and `D`.  You should **replace these with your actual data**. 
*   **Comments:** The code includes comments to explain each section.
*   **Solver:** The code uses the `cbc` solver, but you can change this to another solver you have installed (e.g., 'gurobi', 'glpk', etc.).
*   **Output:** The code will print the solver's status (to check if a solution was found) and the optimal power output for each unit at each time period.

This code provides a solid foundation for modeling your power plant scheduling problem in Pyomo.  Remember to input your actual data and choose a suitable solver to obtain meaningful results. 


## 4. Input problem data and try running the generated code

In [181]:
import pyomo.environ as pyo

# Sample Data (Replace with actual data)
SU = [10324, 5678, 7802, 12899, 4596, 9076]  
SD = [2673, 5893, 982, 6783, 2596, 3561]
FC = [100, 80, 90, 120, 70, 110] # Sample data
VC = { (n,t): 0.5 + 0.1*n + 0.05*t for n in range(1,7) for t in range(1,16)} # Sample data
P_min = [50, 30, 40, 60, 20, 80] # Sample data
P_max = [200, 150, 180, 250, 100, 300] # Sample data
R_up = [40, 30, 35, 50, 25, 60] # Sample data
R_down = [40, 30, 35, 50, 25, 60] # Sample data
D = [400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 1050, 1100] # Sample data

# Model Initialization
model = pyo.ConcreteModel()

# Sets
model.T = pyo.RangeSet(1, 15) # Time periods
model.N = pyo.RangeSet(1, 6) # Power units

# Parameters
model.SU = pyo.Param(model.N, initialize=dict(enumerate(SU, 1))) # Startup cost
model.SD = pyo.Param(model.N, initialize=dict(enumerate(SD, 1))) # Shutdown cost
model.FC = pyo.Param(model.N, initialize=dict(enumerate(FC, 1))) # Fixed cost
model.VC = pyo.Param(model.N, model.T, initialize=VC) # Variable cost
model.P_min = pyo.Param(model.N, initialize=dict(enumerate(P_min, 1))) # Minimum power output
model.P_max = pyo.Param(model.N, initialize=dict(enumerate(P_max, 1))) # Maximum power output
model.R_up = pyo.Param(model.N, initialize=dict(enumerate(R_up, 1))) # Ramp-up rate
model.R_down = pyo.Param(model.N, initialize=dict(enumerate(R_down, 1))) # Ramp-down rate
model.D = pyo.Param(model.T, initialize=dict(enumerate(D, 1))) # Power demand

# Decision Variables
model.P = pyo.Var(model.N, model.T, within=pyo.NonNegativeReals) # Power output
model.U = pyo.Var(model.N, model.T, within=pyo.Binary) # Startup status
model.V = pyo.Var(model.N, model.T, within=pyo.Binary) # Shutdown status

# Objective Function
def objective_rule(model):
    return sum(model.FC[n] * model.U[n,t] + model.VC[n,t] * model.P[n,t] + model.SU[n] * model.U[n,t] + model.SD[n] * model.V[n,t] for n in model.N for t in model.T)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
# Power balance
def power_balance_rule(model, t):
    return sum(model.P[n,t] for n in model.N) >= 1.1 * model.D[t] # 1.1 represents a 10% safety margin
model.power_balance = pyo.Constraint(model.T, rule=power_balance_rule)

# Power output limits
def power_output_limits_rule(model, n, t):
    return (model.P_min[n], model.P[n,t], model.P_max[n])
model.power_output_limits = pyo.Constraint(model.N, model.T, rule=power_output_limits_rule)

# Ramp rate limits
def ramp_up_rule(model, n, t):
    if t > 1:
        return model.P[n,t] - model.P[n,t-1] <= model.R_up[n]
    else:
        return pyo.Constraint.Skip
model.ramp_up = pyo.Constraint(model.N, model.T, rule=ramp_up_rule)

def ramp_down_rule(model, n, t):
    if t > 1:
        return model.P[n,t-1] - model.P[n,t] <= model.R_down[n]
    else:
        return pyo.Constraint.Skip
model.ramp_down = pyo.Constraint(model.N, model.T, rule=ramp_down_rule)

# Startup and Shutdown logic
def startup_shutdown_rule(model, n, t):
    if t > 1:
        return model.U[n,t] - model.V[n,t] == (model.P[n,t] > 0) - (model.P[n,t-1] > 0)
    else:
        return pyo.Constraint.Skip
model.startup_shutdown = pyo.Constraint(model.N, model.T, rule=startup_shutdown_rule)

# Solve the model
solver = pyo.SolverFactory('cbc') # Replace 'cbc' with your preferred solver
results = solver.solve(model)

# Print results
print(results)

# Access and print specific variable values
for t in model.T:
    print(f"Time period {t}:")
    for n in model.N:
        print(f"  Unit {n} power output: {pyo.value(model.P[n,t])}")

ERROR: Rule failed when generating expression for Constraint startup_shutdown
with index (1, 2): TypeError: unsupported operand type(s) for -:
'InequalityExpression' and 'InequalityExpression'
ERROR: Constructing component 'startup_shutdown' from data=None failed:
        TypeError: unsupported operand type(s) for -: 'InequalityExpression'
        and 'InequalityExpression'


TypeError: unsupported operand type(s) for -: 'InequalityExpression' and 'InequalityExpression'

## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [184]:
print(response.text)

## Mathematical Optimization Model for Power Plant Scheduling

**Parameters:**

*  **T:** Number of time periods (T = 15).
*  **N:** Number of power units (N = 6).
*  **SU_n:** Startup cost for power unit n (€).
*  **SD_n:** Shutdown cost for power unit n (€).
*  **FC_n:** Fixed cost for running power unit n (€).
*  **VC_{n,t}:** Variable cost per unit output for power unit n at time t (€/MWh).
*  **P_min_n:** Minimum power output for power unit n (MW).
*  **P_max_n:** Maximum power output for power unit n (MW).
*  **R_up_n:** Maximum power increment for power unit n from one time period to the next (MW).
*  **R_down_n:** Maximum power decrement for power unit n from one time period to the next (MW).
*  **D_t:** Total power demand at time t (MW).

**Decision Variables:**

*  **P_{n,t}:** Power output of unit n at time t (MW).
*  **U_{n,t}:** Binary variable indicating if unit n is started up at time t (1 if started, 0 otherwise).
*  **V_{n,t}:** Binary variable indicating if unit n is 

In [186]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (Replace with actual data)
SU = [10324, 5678, 7802, 12899, 4596, 9076]  
SD = [2673, 5893, 982, 6783, 2596, 3561]
FC = [100, 80, 90, 120, 70, 110] # Sample data
VC = { (n,t): 0.5 + 0.1*n + 0.05*t for n in range(1,7) for t in range(1,16)} # Sample data
P_min = [50, 30, 40, 60, 20, 80] # Sample data
P_max = [200, 150, 180, 250, 100, 300] # Sample data
R_up = [40, 30, 35, 50, 25, 60] # Sample data
R_down = [40, 30, 35, 50, 25, 60] # Sample data
D = [400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 1050, 1100] # Sample data

# Model Initialization
model = pyo.ConcreteModel()

# Sets
model.T = pyo.RangeSet(1, 15) # Time periods
model.N = pyo.RangeSet(1, 6) # Power units

# Parameters
model.SU = pyo.Param(model.N, initialize=dict(enumerate(SU, 1))) # Startup cost
model.SD = pyo.Param(model.N, initialize=dict(enumerate(SD, 1))) # Shutdown cost
model.FC = pyo.Param(model.N, initialize=dict(enumerate(FC, 1))) # Fixed cost
mod